

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CONTEXTUAL_SPELL_CHECKER.ipynb)




# **Spell checking for clinical documents**

To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

In [ ]:
import os
import json

with open('/content/spark_nlp_for_healthcare.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

Install dependencies

In [ ]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

Import dependencies into Python and start the Spark session

In [ ]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

## 2. Select the NER model and construct the pipeline

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = RecursiveTokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token') \
    .setPrefixes(["\"", "(", "[", "\n"]) \
    .setSuffixes([".", ",", "?", ")","!", "‘s"])

spell_model = ContextSpellCheckerModel.pretrained('spellcheck_clinical', 'en', 'clinical/models') \
    .setInputCols('token') \
    .setOutputCol('corrected')

finisher = Finisher().setInputCols('corrected')

light_pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    spell_model,
    finisher
])

full_pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    spell_model
])

empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = full_pipeline.fit(empty_df)
light_pipeline_model = LightPipeline(light_pipeline.fit(empty_df))

## 3. Create example inputs

In [ ]:
# Enter examples as strings in this array
input_list = [
    "The pateint is a 5-mont-old infnt who presented initially on Monday with a cold, cugh, and runny nse for 2 days. Mom states she had no fevr. Her appetite was good but she was spitting up a lot. She had no difficulty breathin and her cough was described as dry and hacky. At that time, pysicl exam showed a right TM, which was red. Left TM was okay. She was fairly congsted but looked happy and playful. She was started on Amxil and Aldx and we told to recheck in 2 weaks to recheck her ear. Mom returned to clinic again today because she got much worse ovrnght. She was having dificlty breathing. She was much more congested and her apetit had decrsed significantly today. She also spked a tempratre yesterday of 102.6 and always hvng trouble sleping scondry to congestion."
]

## 4. Use the pipeline to create outputs

Full Pipeline

In [ ]:
df = spark.createDataFrame(pd.DataFrame({'text': input_list}))
result = pipeline_model.transform(df)

Light Pipeline

In [ ]:
# Light pipelines use plain string inputs instead of data frame inputs
light_result = light_pipeline_model.annotate(input_list[0])

## 5. Visualize results

Visualize comparison as dataframe

In [ ]:
exploded = F.explode(F.arrays_zip('token.result', 'corrected.result'))
select_expression_0 = F.expr("cols['0']").alias("original")
select_expression_1 = F.expr("cols['1']").alias("corrected")
result.select(exploded.alias("cols")) \
    .select(select_expression_0, select_expression_1).show(truncate=False)

Vizualise light pipeline and finished result

In [ ]:
# This finished result does not need parsing and can directly be used in any
# other task
light_result['corrected']